In [ ]:
import threading
import time

In [ ]:
class ReadWriteLock:
    def __init__(self):
        self.readers_count = 0
        self.writers_count = 0
        self.readers_count_lock = threading.Lock()
        self.resource_lock = threading.Lock()

    def acquire_read(self):
        with self.readers_count_lock:
            self.readers_count += 1
            if self.readers_count == 1:
                self.resource_lock.acquire()

    def release_read(self):
        with self.readers_count_lock:
            self.readers_count -= 1
            if self.readers_count == 0:
                self.resource_lock.release()

    def acquire_write(self):
        with self.resource_lock:
            self.writers_count += 1
            while self.readers_count > 0:
                # Wait for all readers to finish
                pass

    def release_write(self):
        with self.resource_lock:
            self.writers_count -= 1

In [ ]:
# Timer function
def stop_all_processes(lock, threads):
    print("Timer expired. Stopping all processes.")
    
    # Release any locks to allow threads to exit gracefully
    lock.release_read()
    lock.release_write()

    # Interrupt all threads to stop them
    for thread in threads:
        thread.join(timeout=0.1)  # Wait for the thread to finish
        raise SystemExit(0)  # Interrupt the thread

In [ ]:
def reader(lock, reader_id):
    while True:
        time.sleep(1)  # Simulate reading
        lock.acquire_read()
        print(f"Reader {reader_id} is reading.")
        lock.release_read()

def writer(lock, writer_id, stopp_event):
    while True:
        # global stopp_event
        if stopp_event <= 0:
            break
        time.sleep(2)  # Simulate writing
        lock.acquire_write()
        print(f"Writer {writer_id} is writing.")
        lock.release_write()

In [2]:
# Example usage:      

if __name__ == "__main__":
    stopp_event = 5
    rw_lock = ReadWriteLock()

    # Create reader and writer threads
    readers = [threading.Thread(target=reader, args=(rw_lock, i)) for i in range(3)]
    writers = [threading.Thread(target=writer, args=(rw_lock, i, stopp_event)) for i in range(2)]

    # Start threads
    for r in readers:
        r.start()

    for w in writers:
        w.start()

    # Set up a timer to stop all processes after a specified duration
    timer_duration = 4  # seconds
    timer = threading.Timer(timer_duration, stop_all_processes, args=(rw_lock, readers + writers))
    timer.start()

    # Wait for the timer to expire
    timer.join()

Reader 0 is reading.
Reader 2 is reading.
Reader 1 is reading.
Writer 0 is writing.
Reader 2 is reading.
Reader 0 is reading.
Reader 1 is reading.
Writer 1 is writing.
Reader 0 is reading.Reader 1 is reading.
Reader 2 is reading.

Writer 0 is writing.
Timer expired. Stopping all processes.
Writer 1 is writing.
Reader 0 is reading.
Reader 2 is reading.
Reader 1 is reading.


Reader 1 is reading.
Reader 0 is reading.
Reader 2 is reading.
Writer 0 is writing.
Writer 1 is writing.
Reader 1 is reading.
Reader 0 is reading.
Reader 2 is reading.
Reader 0 is reading.Reader 1 is reading.

Reader 2 is reading.
Writer 0 is writing.
Writer 1 is writing.
Reader 1 is reading.
Reader 0 is reading.
Reader 2 is reading.
Reader 1 is reading.
Reader 0 is reading.
Reader 2 is reading.
Writer 0 is writing.
Writer 1 is writing.
Reader 1 is reading.
Reader 2 is reading.
Reader 0 is reading.
Reader 1 is reading.
Reader 0 is reading.
Reader 2 is reading.
Writer 0 is writing.
Writer 1 is writing.
Reader 1 is reading.
Reader 2 is reading.
Reader 0 is reading.
Reader 1 is reading.
Reader 0 is reading.
Reader 2 is reading.
Writer 0 is writing.
Writer 1 is writing.
Reader 1 is reading.
Reader 0 is reading.
Reader 2 is reading.
Reader 1 is reading.
Reader 2 is reading.
Reader 0 is reading.
Writer 0 is writing.
Writer 1 is writing.
Reader 1 is reading.
Reader 0 is reading.
Reader 2 is r